<a href="https://colab.research.google.com/github/sophiagemanuel/Deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Drop the non-beneficial ID columns, 'EIN'
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [15]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
df_application_type = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(df_application_type[df_application_type < 100].index)
application_types_to_replace

['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [16]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [17]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)


# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype='float')
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=41)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**First Optimization Method** *Creating a third hidden layer*


In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Creating a model and adding the layers in one def.
features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()
# First hidden layer
first_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=features, activation='relu')
nn.add(first_hidden_layer)
# Second hidden layer
second_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu')
nn.add(second_hidden_layer)

# Third hidden layer
third_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu')
nn.add(third_hidden_layer)

# Output layer
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')
nn.add(output_layer)

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3600      
                                                                 
 dense_9 (Dense)             (None, 100)               8100      
                                                                 
 dense_10 (Dense)            (None, 40)                4040      
                                                                 
 dense_11 (Dense)            (None, 1)                 41        
                                                                 
Total params: 15781 (61.64 KB)
Trainable params: 15781 (61.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Compile the model
optimized_model = create_optimized_model()
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
# Train the model
#Second Optimization: Adding more epochs and batch size.
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5690 - accuracy: 0.7224
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7308
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7317
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7343
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7346
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5467 - accuracy: 0.7362
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7357
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7361
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5804 - accuracy: 0.7235 - 600ms/epoch - 2ms/step
Loss: 0.58038729429245, Accuracy: 0.723498523235321


**Second Optimization Method** *Adding more Epochs*

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Creating a model and adding the layers in one def.
features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()
# First hidden layer
first_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=features, activation='relu')
nn.add(first_hidden_layer)
# Second hidden layer
second_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu')
nn.add(second_hidden_layer)

# Third hidden layer
third_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu')
nn.add(third_hidden_layer)

# Output layer
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')
nn.add(output_layer)

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 80)                3600      
                                                                 
 dense_17 (Dense)            (None, 100)               8100      
                                                                 
 dense_18 (Dense)            (None, 40)                4040      
                                                                 
 dense_19 (Dense)            (None, 1)                 41        
                                                                 
Total params: 15781 (61.64 KB)
Trainable params: 15781 (61.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# Compile the model
optimized_model = create_optimized_model()
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5687 - accuracy: 0.7241
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7318
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7326
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7342
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7345
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7354
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7351
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5456 - accuracy: 0.7354
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7357
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accura

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6025 - accuracy: 0.7241 - 886ms/epoch - 3ms/step
Loss: 0.6025256514549255, Accuracy: 0.7240816354751587


**Third Optimization** *Batch Size*

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Creating a model and adding the layers in one def.
features = len(X_train_scaled[0])

hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()
# First hidden layer
first_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=features, activation='relu')
nn.add(first_hidden_layer)
# Second hidden layer
second_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu')
nn.add(second_hidden_layer)

# Third hidden layer
third_hidden_layer = tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu')
nn.add(third_hidden_layer)

# Output layer
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')
nn.add(output_layer)

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 80)                3600      
                                                                 
 dense_25 (Dense)            (None, 100)               8100      
                                                                 
 dense_26 (Dense)            (None, 40)                4040      
                                                                 
 dense_27 (Dense)            (None, 1)                 41        
                                                                 
Total params: 15781 (61.64 KB)
Trainable params: 15781 (61.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Compile the model
optimized_model = create_optimized_model()
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200, batch_size=55)

Epoch 1/200
468/468 [==============================] - 5s 4ms/step - loss: 0.5694 - accuracy: 0.7241
Epoch 2/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5544 - accuracy: 0.7315
Epoch 3/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5520 - accuracy: 0.7336
Epoch 4/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5487 - accuracy: 0.7334
Epoch 5/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5481 - accuracy: 0.7339
Epoch 6/200
468/468 [==============================] - 3s 6ms/step - loss: 0.5467 - accuracy: 0.7332
Epoch 7/200
468/468 [==============================] - 2s 5ms/step - loss: 0.5464 - accuracy: 0.7349
Epoch 8/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5457 - accuracy: 0.7354
Epoch 9/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5451 - accuracy: 0.7359
Epoch 10/200
468/468 [==============================] - 2s 4ms/step - loss: 0.5443 - accura

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5902 - accuracy: 0.7247 - 1s/epoch - 4ms/step
Loss: 0.5902111530303955, Accuracy: 0.7246647477149963
